**Cloning repository**

In [1]:
!git clone https://github.com/Edresson/TTS-Conv.git
import os
os.chdir('TTS-Conv')


fatal: destination path 'TTS-Conv' already exists and is not an empty directory.


**Import modules**

In [0]:
from hyperparams import Hyperparams as hp
import numpy as np
import tensorflow as tf
from train import Graph
from utils import *
from scipy.io.wavfile import write
from tqdm import tqdm
from librosa import  display
from data_load import text_normalize,load_vocab
from IPython.display import Audio



**Download Weights**




In [3]:

!wget -c -q --show-progress -O ./saver-text.zip https://www.dropbox.com/s/oeafuy4yp7nqj5y/saver-text.zip?dl=0
!ls
!unzip saver-text.zip
  

 data_load.py				       __pycache__
 environment.yml			       README.md
'evaluate-objectives (cópia 1).py'	       sample-208-10w.wav
 evaluate-objectives.py			       saver-text
 get_best_checkpoint.py			       saver-text.zip
 harvard_sentences.txt			       spectogram-extract.py
 hyperparams.py				       synthesize-org.py
 LICENSE				       synthesize.py
 modules.py				       test_file.py
 networks.py				       test_phrase_to_phoneme.py
 PETRUS					       test_word.py
 phonetically-balanced-sentences-phoneme.txt   train.py
 phonetically-balanced-sentences.txt	       utils.py
 prepo.py
Archive:  saver-text.zip
replace saver-text/mel2linear/saver.data-00000-of-00001? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: saver-text/mel2linear/saver.data-00000-of-00001  
  inflating: saver-text/mel2linear/saver.index  
  inflating: saver-text/mel2linear/saver.meta  
  inflating: saver-text/text2mel/saver.data-00000-of-00001  
  inflating: saver-text/text2mel/saver.index  
  inflating: saver-

**Restore Model**

In [4]:
# Load graph
g = Graph(mode="synthesize"); print("Graph loaded")

sess = tf.Session()

sess.run(tf.global_variables_initializer())

# Restore parameters
var_list = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'Text2Mel')
saver1 = tf.train.Saver(var_list=var_list)
saver1.restore(sess, os.path.join('saver-text','text2mel','saver'))
print("Text2Mel Restored!")

var_list = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'SSRN') + tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, 'gs')
saver2 = tf.train.Saver(var_list=var_list)
saver2.restore(sess, os.path.join('saver-text','mel2linear','saver'))
print("SSRN Restored!")

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.conv1d instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use keras.layers.conv2d_transpose instead.
Graph loaded
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from saver-text/text2mel/saver
Text2Mel Restored!
INFO:tensorflow:Restoring parameters from saver-text/mel2linear/saver
SSRN Restored!


**Synthesize**


In [18]:

import IPython
frase = input("Digite o frase: ")
print('input text: ',frase)
frase = '1 '+frase
#normalize remove inavalid characters
frase = text_normalize(frase.split(" ", 1)[-1]).strip() + "E" # text normalization, E: EOS

print('normalized text:',frase)
      
char2idx, idx2char = load_vocab()
      
#convert characters to numbers
text = np.zeros((1, hp.max_N), np.int32)#hp.max_N = 128, is the max number for characters 
text[0, :len(frase)] = [char2idx[char] for char in frase]

print('converted text:',text)

L = text
# Feed Forward
## mel
Y = np.zeros((len(L), hp.max_T, hp.n_mels), np.float32)
prev_max_attentions = np.zeros((len(L),), np.int32)
for j in tqdm(range(hp.max_T)):
    _gs, _Y, _max_attentions, _alignments = sess.run([g.global_step, g.Y, g.max_attentions, g.alignments], {g.L: L,g.mels: Y, g.prev_max_attentions: prev_max_attentions})
    Y[:, j, :] = _Y[:, j, :]
    prev_max_attentions = _max_attentions[:, j]

    

# Get magnitude
Z = sess.run(g.Z, {g.Y: Y})

# Generate wav files
for i, mag in enumerate(Z):
    print("Working on file", i+1)
    wav = spectrogram2wav(mag)
    #save for frase.wav
    write("frase.wav", hp.sr, wav) 
    #play
    IPython.display.display(IPython.display.Audio( wav, rate=hp.sr))


Digite o frase: Oii


  1%|▏         | 3/210 [00:00<00:07, 27.83it/s]

input text:  Oii
Oii
normalized text: oiiE
converted text: [[17 11 11  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0]]


100%|██████████| 210/210 [00:05<00:00, 37.34it/s]


Working on file 1


In [8]:
!ls

 data_load.py				       prepo.py
 environment.yml			       __pycache__
'evaluate-objectives (cópia 1).py'	       README.md
 evaluate-objectives.py			       sample-208-10w.wav
 frase.wav				       saver-text
 get_best_checkpoint.py			       saver-text.zip
 harvard_sentences.txt			       spectogram-extract.py
 hyperparams.py				       synthesize-org.py
 LICENSE				       synthesize.py
 modules.py				       test_file.py
 networks.py				       test_phrase_to_phoneme.py
 PETRUS					       test_word.py
 phonetically-balanced-sentences-phoneme.txt   train.py
 phonetically-balanced-sentences.txt	       utils.py
